## Import Libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
#Create a path to where your data is stored.
path = r'C:\Users\shaiv\Downloads\ClimateWins'

In [5]:
# Import unscaled weather data
df_unscaled = pd.read_csv(os.path.join(path, '02 Data', 'weather prediction dataset processed.csv'))

In [6]:
# Import pleasant weather data
df_pleasant = pd.read_csv(os.path.join(path, '02 Data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

## Clean Data for Deep Learning

In [8]:
df_unscaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [11]:
df_pleasant.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
# Drop date and month columns from df_unscaled
df_unscaled = df_unscaled.drop(columns=['DATE', 'MONTH'], axis=1)
df_unscaled.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [18]:
# Drop date column from df_pleasant
df_pleasant = df_pleasant.drop(columns=['DATE'], axis=1)
df_pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# Drop all Gdansk, Roma,Tours columns from df_unscaled since they are not included in pleasant weather data
# We know this from previous exercises using the same data
cols_to_drop = [col for col in df_unscaled.columns if col.startswith(('GDANSK', 'ROMA', 'TOURS'))]
df_unscaled = df_unscaled.drop(columns=cols_to_drop)

In [22]:
df_unscaled.columns

Index(['BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean',
       'BASEL_temp_min',
       ...
       'VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure',
       'VALENTIA_global_radiation', 'VALENTIA_precipitation',
       'VALENTIA_snow_depth', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=147)

In [24]:
# Trying to find out all the different measurement types for each location

# Extract location names 
locations = set([col.split('_')[0] for col in df_unscaled.columns])

# Create a dictionary to store measurement counts for each location
measurement_counts = {location: {} for location in locations}

# Count occurrences of each measurement type for each location
for col in df_unscaled.columns:
    parts = col.split('_') 
    location = parts[0] 
    measurement = '_'.join(parts[1:])  # Join remaining parts if there are more than two

    if measurement not in measurement_counts[location]:
        measurement_counts[location][measurement] = 1
    else:
        measurement_counts[location][measurement] += 1

# Print the measurement counts for each location
for location, measurements in measurement_counts.items():
    print(f"Location: {location}")
    for measurement, count in measurements.items():
        print(f"  - {measurement}: {count}")
    print()

Location: BELGRADE
  - cloud_cover: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MADRID
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: KASSEL
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: VALENTIA
  - cloud_cover: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MAASTRICHT
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: DUSSELDORF
  - cloud_cover: 1
  - 

Most measurements at a single location is 11 and they are:
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

- Missing
- Maastricht: snow_depth
- Madrid: snow_depth
- Heathrow: wind_speed measurements
- Kassel: cloud_cover, snow_depth
- Belgrade: wind_speed, snow_depth
- Valentia: wind_speed
- Budapest: wind_speed, snow_depth
- Ljubljana: snow_depth
- Sonnblick: snow_depth
- Stockholm: wind_speed, humidity, snow_depth
- Debilt: snow_depth
- Munchenb wind_speed, pressure
s missing 

In [27]:
# Drop columns for wind_speed and snow_depth measurements since they are missing so often

# Create a list of columns to drop
cols_to_drop = [col for col in df_unscaled.columns if col.endswith(('wind_speed', 'snow_depth'))]

# Dropping
df_unscaled = df_unscaled.drop(columns=cols_to_drop)

In [29]:
df_unscaled.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [31]:
# There are missing measurements for Kassel's cloud cover, Stockholm's humidity, and Munchenb's pressure
# We know that Ljubljana is near Kassel, Sonnblick is near Munchenb, and Olso is close enough to Stockholm

# Define relationships between locations
location_pairs = {
    'KASSEL': 'LJUBLJANA',
    'STOCKHOLM': 'OSLO',
    'MUNCHENB': 'SONNBLICK'
}

# Define the desired order of measurements
measurement_order = ['cloud_cover', 'humidity', 'pressure', 'global_radiation', 
                     'precipitation', 'sunshine', 'temp_mean', 'temp_min', 'temp_max']

# Function to fill missing values and insert in correct position
def fill_missing_values(df_unscaled, location, measurement, neighbor):
    """
    Fills missing values for a given location and measurement using data from a neighbor location.
    Inserts the new column in the correct position based on the measurement order.

    Args:
        df_unscaled: The DataFrame containing the weather data.
        location: The location with missing values.
        measurement: The measurement with missing values.
        neighbor: The neighboring location to use for filling.

    Returns:
        The updated DataFrame with filled missing values and columns in the correct order.
    """
    source_col = f'{neighbor}_{measurement}'
    target_col = f'{location}_{measurement}'

    # Determine the insertion index 
    if measurement == measurement_order[0]:  # If it's the first measurement for the location
        # Find the index of the first column for the location (or 0 if no location columns exist)
        location_columns = [col for col in df_unscaled.columns if col.startswith(location)]
        if location_columns:
            insert_index = df_unscaled.columns.get_loc(location_columns[0]) 
        else:
            insert_index = 0
    else:
        insert_index = df_unscaled.columns.get_loc(f'{location}_{measurement_order[measurement_order.index(measurement) - 1]}') + 1 

    # Create the new column with missing values and insert it at the correct position
    df_unscaled.insert(insert_index, target_col, np.nan) 

    # Fill missing values in the new column
    df_unscaled[target_col].fillna(df_unscaled[source_col], inplace=True) 

    return df_unscaled

# Fill missing values for each location and measurement
for location, neighbor in location_pairs.items():
    for measurement in measurement_order:
        if f'{location}_{measurement}' not in df_unscaled.columns:  # Check if column already exists
            df_unscaled = fill_missing_values(df_unscaled, location, measurement, neighbor)

# Checking new columns for existance and location
selected_columns = [col for col in df_unscaled.columns if col.startswith(('KASSEL', 'STOCKHOLM', 'MUNCHENB'))]
print(df_unscaled[selected_columns])

       KASSEL_cloud_cover  KASSEL_humidity  KASSEL_pressure  \
0                     8.0             0.82           1.0094   
1                     6.0             0.86           1.0086   
2                     8.0             0.91           1.0129   
3                     6.0             0.87           1.0290   
4                     7.0             0.86           1.0262   
...                   ...              ...              ...   
22945                 4.0             0.77           1.0161   
22946                 3.0             0.77           1.0161   
22947                 3.0             0.77           1.0161   
22948                 3.0             0.77           1.0161   
22949                 3.0             0.77           1.0161   

       KASSEL_global_radiation  KASSEL_precipitation  KASSEL_sunshine  \
0                         0.28                  0.48              1.6   
1                         0.12                  0.27              0.0   
2                       

C:\Users\shaiv\AppData\Local\Temp\ipykernel_19384\1246060646.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_unscaled[target_col].fillna(df_unscaled[source_col], inplace=True)
C:\Users\shaiv\AppData\Local\Temp\ipykernel_19384\1246060646.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

In [33]:
# Checking new columns for existance and location
selected_columns = [col for col in df_unscaled.columns if col.startswith(('MUNCHENB'))]
print(df_unscaled[selected_columns])

       MUNCHENB_cloud_cover  MUNCHENB_humidity  MUNCHENB_pressure  \
0                         5               0.67             1.0304   
1                         6               0.72             1.0292   
2                         6               0.91             1.0320   
3                         6               0.90             1.0443   
4                         5               0.85             1.0430   
...                     ...                ...                ...   
22945                     2               0.76             1.0263   
22946                     6               0.70             1.0263   
22947                     7               0.64             1.0263   
22948                     6               0.75             1.0263   
22949                     5               0.83             1.0263   

       MUNCHENB_global_radiation  MUNCHENB_precipitation  MUNCHENB_sunshine  \
0                           0.20                    0.10                0.0   
1            

In [35]:
df_unscaled.shape

(22950, 135)

In [37]:
df_pleasant.shape

(22950, 15)

In [39]:
# Export cleaned weather data
df_unscaled.to_csv(os.path.join(path, '02 Data','weather_clean.csv'), index=False)

In [41]:
df_unscaled.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


## Reshape Data for Deep Learning

In [41]:
# Rename df's
X = df_unscaled
y = df_pleasant

In [42]:
# Convert df's to arrays
X = np.array(X)
y = np.array(y)
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [43]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [44]:
# Reshaping X as a 3D object
X = X.reshape(-1,15,9)

In [45]:
X.shape

(22950, 15, 9)

In [46]:
y.shape

(22950, 15)

## Split Data into Training and Testing Sets

In [48]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

## Create Keras Model

In [58]:
epochs = 20
batch_size = 16
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16)             │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,919 (7.50 KB)

 Trainable params: 1,919 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

## Compile & Run RNN Model

In [63]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [65]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.0534 - loss: 9.9711 - val_accuracy: 0.0423 - val_loss: 8.8323
Epoch 2/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.2037 - loss: 9.9955 - val_accuracy: 0.2591 - val_loss: 8.7602
Epoch 3/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.2715 - loss: 9.8974 - val_accuracy: 0.2530 - val_loss: 8.8471
Epoch 4/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.2725 - loss: 10.0933 - val_accuracy: 0.2480 - val_loss: 8.9628
Epoch 5/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.2824 - loss: 9.8371 - val_accuracy: 0.2524 - val_loss: 9.1270
Epoch 6/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.3228 - loss: 9.9842 - val_accuracy: 0.2602 - val_loss: 9.2240
Epoch 7/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4205 - loss: 9.9888 - val_accuracy: 0.6276 - val_loss: 9.4310
Epoch 8/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4952 

## Confusion Matrix of Model Results

In [67]:
# Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [68]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [69]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Pred        BASEL  MADRID
True                     
BASEL        3682       0
BELGRADE     1089       3
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       82       0
KASSEL         11       0
LJUBLJANA      60       1
MAASTRICHT      9       0
MADRID        453       5
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


## Recreating Steps above with Adjusted Hyperparameters

In [71]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [72]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,871 (22.93 KB)

 Trainable params: 5,871 (22.93 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [74]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.1774 - loss: 11.0524 - val_accuracy: 0.2668 - val_loss: 9.3651
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.2738 - loss: 10.7321 - val_accuracy: 0.2935 - val_loss: 9.0615
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.2859 - loss: 10.9142 - val_accuracy: 0.2722 - val_loss: 9.3915
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.4199 - loss: 10.9563 - val_accuracy: 0.6415 - val_loss: 9.7793
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.5903 - loss: 11.2418 - val_accuracy: 0.6434 - val_loss: 10.2118
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.6360 - loss: 11.3560 - val_accuracy: 0.6420 - val_loss: 10.4268
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.6349 - loss: 11.5552 - val_accuracy: 0.6398 - val_loss: 10.7167
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy

In [75]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [76]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Pred        BASEL  MADRID
True                     
BASEL        3681       1
BELGRADE     1092       0
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       81       1
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        453       5
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


##### Accuracy appears to decrease while loss increase. 

## Adjusting Hyperparameters & Changing Activation

In [79]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [80]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,919 (77.81 KB)

 Trainable params: 19,919 (77.81 KB)

 Non-trainable params: 0 (0.00 B)

In [81]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [82]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.0442 - loss: 25.3272 - val_accuracy: 0.3200 - val_loss: 21.0664
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.2092 - loss: 24.9161 - val_accuracy: 0.0047 - val_loss: 23.9220
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.0775 - loss: 24.9362 - val_accuracy: 0.1112 - val_loss: 24.1376
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.1117 - loss: 24.5415 - val_accuracy: 0.0326 - val_loss: 25.5545
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.0792 - loss: 24.9849 - val_accuracy: 0.1145 - val_loss: 23.3130
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.1363 - loss: 24.7901 - val_accuracy: 0.1833 - val_loss: 25.8125
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.1403 - loss: 24.4854 - val_accuracy: 0.0166 - val_loss: 24.2806
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - accu

In [83]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [84]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Pred        BASEL  DEBILT  KASSEL  MAASTRICHT
True                                         
BASEL         136     111    3339          96
BELGRADE        0       0    1092           0
BUDAPEST        0       0     214           0
DEBILT          0       0      82           0
DUSSELDORF      0       0      29           0
HEATHROW        0       0      82           0
KASSEL          0       0      11           0
LJUBLJANA       0       0      61           0
MAASTRICHT      0       0       9           0
MADRID          0       0     455           3
MUNCHENB        0       0       8           0
OSLO            0       0       5           0
STOCKHOLM       0       0       4           0
VALENTIA        0       0       1           0


## Adjusting Hyperparameters, Adding Convolution and Pooling Layers, and Changing Optimizer

In [86]:
epochs = 25
batch_size = 16
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [87]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 8)              │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 831 (3.25 KB)

 Trainable params: 831 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [89]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.0788 - loss: 25.9106 - val_accuracy: 0.0837 - val_loss: 26.1220
Epoch 2/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1079 - loss: 24.1599 - val_accuracy: 0.1194 - val_loss: 30.4872
Epoch 3/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0800 - loss: 21.9247 - val_accuracy: 0.1300 - val_loss: 21.9562
Epoch 4/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0629 - loss: 24.7457 - val_accuracy: 0.0523 - val_loss: 23.8145
Epoch 5/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0605 - loss: 22.7261 - val_accuracy: 0.1239 - val_loss: 22.5157
Epoch 6/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.1057 - loss: 22.1063 - val_accuracy: 0.1795 - val_loss: 31.1499
Epoch 7/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0799 - loss: 22.0742 - val_accuracy: 0.1673 - val_loss: 32.3384
Epoch 8/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.041

In [90]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [91]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1


## Returning to Original Keras Setup with Adjusted Hyperparameters

In [93]:
epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [94]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 4)              │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 299 (1.17 KB)

 Trainable params: 299 (1.17 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [96]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.1036 - loss: 8.7593 - val_accuracy: 0.0798 - val_loss: 8.5958
Epoch 2/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.1706 - loss: 9.2238 - val_accuracy: 0.3114 - val_loss: 9.3536
Epoch 3/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.2611 - loss: 9.9660 - val_accuracy: 0.3106 - val_loss: 10.0277
Epoch 4/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.2823 - loss: 10.6578 - val_accuracy: 0.3093 - val_loss: 10.6448
Epoch 5/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.2809 - loss: 11.1865 - val_accuracy: 0.3179 - val_loss: 11.2295
Epoch 6/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.2800 - loss: 11.7772 - val_accuracy: 0.3111 - val_loss: 11.7664
Epoch 7/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.3321 - loss: 12.3614 - val_accuracy: 0.3076 - val_loss: 12.1984
Epoch 8/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.3430 

In [97]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [98]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        1469    2213
BELGRADE      960     132
BUDAPEST      198      16
DEBILT         82       0
DUSSELDORF     25       4
HEATHROW       66      16
KASSEL         11       0
LJUBLJANA      45      16
MAASTRICHT      3       6
MADRID        160     298
MUNCHENB        6       2
OSLO            3       2
STOCKHOLM       4       0
VALENTIA        0       1


## CNN Model

In [100]:
epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [101]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [102]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [103]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1318 - loss: 15968.0674
Epoch 2/10
4303/4303 - 7s - 2ms/step - accuracy: 0.1309 - loss: 160778.0938
Epoch 3/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1350 - loss: 530804.0625
Epoch 4/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1281 - loss: 1205138.5000
Epoch 5/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1290 - loss: 2224027.5000
Epoch 6/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1311 - loss: 3659521.7500
Epoch 7/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1355 - loss: 5708805.5000
Epoch 8/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1336 - loss: 8256710.0000
Epoch 9/10
4303/4303 - 7s - 2ms/step - accuracy: 0.1366 - loss: 11497005.0000
Epoch 10/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1350 - loss: 15683079.0000


In [104]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [105]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BUDAPEST  DEBILT  HEATHROW  KASSEL  LJUBLJANA  MAASTRICHT  MADRID  \
True                                                                            
BASEL           1389     196        46      60        163          12    1498   
BELGRADE         436       4         1       0         37           0     606   
BUDAPEST          52       0         0       0          4           0     158   
DEBILT            25       0         0       0          0           0      57   
DUSSELDORF         3       0         0       0          0           0      26   
HEATHROW           7       0         0       0          0           0      75   
KASSEL             6       0         0       0          0           0       5   
LJUBLJANA         14       0         0       0          0           0      47   
MAASTRICHT         5       0         0       0          0           0       4   
MADRID            28       5         0       0          1           

##### There is already significant improvement with the number of stations identified using the CNN model compared to the RNN model

## CNN Model Retest

In [108]:
epochs = 15
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [109]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 14, 16)         │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,991 (7.78 KB)

 Trainable params: 1,991 (7.78 KB)

 Non-trainable params: 0 (0.00 B)

In [110]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [111]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 5s - 2ms/step - accuracy: 0.1275 - loss: 7921.8081
Epoch 2/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1288 - loss: 77785.0469
Epoch 3/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1312 - loss: 246870.9688
Epoch 4/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1222 - loss: 541670.3125
Epoch 5/15
2152/2152 - 3s - 2ms/step - accuracy: 0.1231 - loss: 967730.4375
Epoch 6/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1267 - loss: 1577445.7500
Epoch 7/15
2152/2152 - 6s - 3ms/step - accuracy: 0.1242 - loss: 2355776.0000
Epoch 8/15
2152/2152 - 6s - 3ms/step - accuracy: 0.1252 - loss: 3332575.2500
Epoch 9/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1208 - loss: 4610706.5000
Epoch 10/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1257 - loss: 6079326.0000
Epoch 11/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1234 - loss: 7825004.0000
Epoch 12/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1203 - loss: 9875189.0000
Epoch 13/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1253 - loss: 12270950.0000
Ep

In [112]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [113]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL          17       112       302      87           5        13     110   
BELGRADE        0         3        12       6           1         3       0   
BUDAPEST        0         0         4       2           1         1       0   
DEBILT          0         0         0       2           1         0       0   
DUSSELDORF      0         0         0       0           0         0       0   
HEATHROW        0         0         0       5           0         1       0   
KASSEL          0         0         0       0           0         0       0   
LJUBLJANA       0         0         0       0           0         0       0   
MAASTRICHT      0         0         0       0           1         0       0   
MADRID          0         0         2      18           0        11       3   
MUNCHENB   

## CNN Final Test

In [115]:
epochs = 15
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh'))

C:\Users\shaiv\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [116]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 14, 16)         │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,991 (7.78 KB)

 Trainable params: 1,991 (7.78 KB)

 Non-trainable params: 0 (0.00 B)

In [117]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 4s - 2ms/step - accuracy: 0.0388 - loss: 26.5047
Epoch 2/15
2152/2152 - 3s - 1ms/step - accuracy: 0.0701 - loss: 24.2299
Epoch 3/15


In [ ]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [ ]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))